[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# *Marsmallow*.

https://marshmallow.readthedocs.io

## El proyecto *Marshmallow*.

In [ ]:
!pip install marshmallow

### La clase ```marshmallow.Schema```.

### Campos.

https://marshmallow.readthedocs.io/en/stable/marshmallow.fields.html

### Validadores.

https://marshmallow.readthedocs.io/en/stable/marshmallow.validate.html

**Ejemplo:**

In [ ]:
from marshmallow import Schema
from marshmallow.fields import String, Int, Float, Bool
from marshmallow.validate import Length, OneOf, Range

In [ ]:
from json import dumps

In [ ]:
from data import carreras

In [ ]:
class AlumnoSchema(Schema):
    cuenta = Int(required=True, validate=Range(min=1000000, max=9999999))
    nombre = String(required=True, validate=Length(min=2, max=50))
    primer_apellido = String(required=True, validate=Length(min=2, max=50))
    segundo_apellido = String(required=False, validate=Length(min=2, max=50))
    carrera = String(required=True, validate=OneOf(carreras))
    semestre = Int(required=True, validate=Range(min=1, max=50))
    promedio = Float(required=True, validate=Range(min=1, max=50))
    al_corriente = Bool(required=True)

In [ ]:
AlumnoSchema().load({
    'cuenta': 1231221, 
    'al_corriente': False, 
    'carrera': 'Arquitectura', 
    'nombre': 'Pedro', 
    'primer_apellido': 'Solis', 
    'promedio': 7.8, 
    'semestre': 3, 
    'segundo_apellido': 'Cabañas'})

In [ ]:
AlumnoSchema().load({
    'cuenta': 1232210, 
    'al_corriente': False, 
    'carrera': 'Arquitectura', 
    'nombre': 'Pedro', 
    'primer_apellido': 'Solis', 
    'promedio': 7.8, 
    'semestre': 3,})

In [ ]:
AlumnoSchema().load({
    'cuenta': 1232210, 
    'al_corriente': False, 
    'carrera': 'Veterinaria', 
    'nombre': 'Pedro', 
    'primer_apellido': 'Solis', 
    'promedio': 7.8, 
    'semestre': 3,})

In [ ]:
AlumnoSchema().load({
    'cuenta': 1232210, 
    'al_corriente': False, 
    'carrera': 'Derecho', 
    'nombre': 'Pedro', 
    'primer_apellido': 'Solis', 
    'promedio': 7.8, 
    'semestre': 3,
    'Género': 'M'})

In [ ]:
from dataclasses import dataclass

In [ ]:
@dataclass
class Alumno:
    cuenta: int
    nombre: str
    primer_apellido: str
    segundo_apellido: str
    carrera: str
    semestre: int
    promedio: float
    al_corriente: bool

In [ ]:
alumno = Alumno(cuenta=1231221, 
    al_corriente=False, 
    carrera='Arquitectura', 
    nombre='Pedro', 
    primer_apellido='Solis', 
    promedio=7.8, 
    semestre=3, 
    segundo_apellido='Cabañas')

In [ ]:
AlumnoSchema().dump(alumno)

## La clase ```marshmallow.Schema.Meta```.


https://marshmallow.readthedocs.io/en/stable/extending.html#custom-class-meta-options

## *Marshmallow-SQLAlchemy*.

https://marshmallow-sqlalchemy.readthedocs.io/en/latest/

### La clase ```marshmallow_sqlalchemy.SQLAlchemySchema```.

In [1]:
import sqlalchemy as sa
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker, relationship, backref

In [2]:
engine = sa.create_engine("sqlite:///data/alumnosql.sqlite")
session = scoped_session(sessionmaker(bind=engine))
Base = declarative_base()

In [3]:
class AlumnosSQL(Base):
    __tablename__ = 'alumno'
    
    cuenta = sa.Column(sa.Integer, primary_key=True)
    nombre = sa.Column(sa.String(50))
    primer_apellido = sa.Column(sa.String(50))
    segundo_apellido = sa.Column(sa.String(50))
    carrera = sa.Column(sa.String(50))
    semestre = sa.Column(sa.Integer)
    promedio = sa.Column(sa.Float)
    al_corriente = sa.Column(sa.Boolean)

In [4]:
Base.metadata.create_all(engine)

In [5]:
from marshmallow_sqlalchemy import SQLAlchemySchema, auto_field

In [6]:
class AlumnoSchema(SQLAlchemySchema):
    class Meta:
        model = AlumnosSQL
        load_instance = True
        
    cuenta = auto_field()
    nombre = auto_field()
    primer_apellido = auto_field()
    segundo_apellido = auto_field()
    carrera = auto_field()
    semestre = auto_field()
    promedio = auto_field()
    al_corriente = auto_field()

In [7]:
alumno = AlumnosSQL(cuenta=12347)

In [8]:
session.add(alumno)
session.commit()

In [9]:
alumno_schema = AlumnoSchema()

In [10]:
alumno_schema.dump(alumno)

{'carrera': None,
 'nombre': None,
 'semestre': None,
 'al_corriente': None,
 'primer_apellido': None,
 'promedio': None,
 'segundo_apellido': None,
 'cuenta': 12347}

In [11]:
nuevo_alumno = alumno_schema.load({
    'cuenta': 12345,
    'al_corriente': False, 
    'carrera': 'Derecho', 
    'nombre': 'Pedro', 
    'primer_apellido': 'Solis', 
    'promedio': 7.8, 
    'semestre': 3,},
    session=session)

In [21]:
nuevo_alumno

In [12]:
session.add(nuevo_alumno)

In [13]:
session.commit()

In [14]:
%load_ext sql

In [15]:
%sql sqlite:///data/alumnosql.sqlite

In [16]:
%%sql 
select * from alumno

 * sqlite:///data/alumnosql.sqlite
Done.


cuenta,nombre,primer_apellido,segundo_apellido,carrera,semestre,promedio,al_corriente
12345,Pedro,Solis,None,Derecho,3,7.8,0
12347,None,None,None,None,None,None,None


### La clase ```marshmallow_sqlalchemy.SQLAlchemyAutoSchema```.

In [17]:
from marshmallow_sqlalchemy import SQLAlchemyAutoSchema

In [18]:
class AlumnoSchema2(SQLAlchemyAutoSchema):
    class Meta:
        model = AlumnosSQL
        include_relationships = True
        load_instance = True

In [19]:
alumno_schema2 = AlumnoSchema2()

In [20]:
alumno_schema2.dump(nuevo_alumno)

{'carrera': 'Derecho',
 'nombre': 'Pedro',
 'semestre': 3,
 'al_corriente': False,
 'primer_apellido': 'Solis',
 'promedio': 7.8,
 'segundo_apellido': None,
 'cuenta': 12345}

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2021.</p>